In [35]:
%%bigquery

BEGIN

CREATE SCHEMA sample_data;

create or replace table `sample_data.smiles` as
(
SELECT product_smiles as smiles,
from `nih-nci-cbiit-chem-prod.savi.plus` 
where rand() < 0.001
);

END

Query complete after 0.00s: 100%|██████████| 1/1 [00:00<00:00, 941.27query/s] 


""


In [30]:
%%bigquery

CREATE SCHEMA exports;

Query complete after 0.00s: 100%|██████████| 1/1 [00:00<00:00, 735.97query/s] 


""


In [19]:
%%timeit -n 2 -r 1

%%bigquery 

create or replace table `exports.savi_fp_json` as
(
SELECT smiles,
`cheminformatics_test.rdkit_pattern_fingerprint_test`(smiles) as fingerprint_json,
`cheminformatics_test.rdkit_molecular_descriptors`(smiles) as molecular_descriptors_json
from `sample_data.smiles_bin_parted` 
)

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1583.55query/s]                        

42 s ± 0 ns per loop (mean ± std. dev. of 1 run, 2 loops each)


In [4]:
%%bigquery 

create or replace table `exports.savi_molecular_descriptors_json` as
(
SELECT smiles,
`cheminformatics_test.rdkit_molecular_descriptors`(smiles) as molecular_descriptors_json
from `sample_data.smiles_bin_parted` 
)

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1337.75query/s]                        


""


In [20]:
%%timeit -n 2 -r 1

%%bigquery

create or replace table `exports.savi_fp_mol_descriptors_clustered`
cluster by num_carbon, num_nitrogen, num_oxygen, num_fluorine
as
(
SELECT smiles,
FROM_HEX(JSON_VALUE(fingerprint_json, '$.fp_pattern_long_as_binary_hex'))  as fp_pattern_as_binary,
FROM_HEX(JSON_VALUE(fingerprint_json, '$.fp_morgan_as_binary_hex'))  as fp_morgan_as_binary,
CAST(JSON_VALUE(fingerprint_json, "$.num_carbon") as INT64) as num_carbon,
CAST(JSON_VALUE(fingerprint_json, "$.num_oxygen") as INT64) as num_oxygen,
CAST(JSON_VALUE(fingerprint_json, "$.num_nitrogen") as INT64) as num_nitrogen,
CAST(JSON_VALUE(fingerprint_json, "$.num_fluorine") as INT64) as num_fluorine,
CAST(JSON_VALUE(fingerprint_json, "$.num_sulfur") as INT64) as num_sulfur,
CAST(JSON_VALUE(molecular_descriptors_json, "$.ExactMolWt") as FLOAT64 ) as exact_mol_wt,
CAST(JSON_VALUE(molecular_descriptors_json, "$.FractionCSP3") as FLOAT64 ) as fraction_csp3,
CAST(JSON_VALUE(molecular_descriptors_json, "$.BalabanJ") as FLOAT64 ) as balabanj,
CAST(JSON_VALUE(molecular_descriptors_json, "$.BertzCT") as FLOAT64 ) as bertzct,
CAST(JSON_VALUE(molecular_descriptors_json, "$.HallKierAlpha") as FLOAT64 ) as hallkieralpha,
CAST(JSON_VALUE(molecular_descriptors_json, "$.HeavyAtomMolWt") as FLOAT64 ) as heavy_atom_mol_wt,
CAST(JSON_VALUE(molecular_descriptors_json, "$.MaxAbsPartialCharge") as FLOAT64 ) as max_abs_partial_charge,
CAST(JSON_VALUE(molecular_descriptors_json, "$.MaxPartialCharge") as FLOAT64 ) as max_partial_charge,
CAST(JSON_VALUE(molecular_descriptors_json, "$.MolLogP") as FLOAT64 ) as mol_logp,
CAST(JSON_VALUE(molecular_descriptors_json, "$.MolMR") as FLOAT64 ) as mol_mr,
CAST(JSON_VALUE(molecular_descriptors_json, "$.HeavyAtomCount") as INT64 ) as heavy_atom_count,
CAST(JSON_VALUE(molecular_descriptors_json, "$.NumAliphaticRings") as INT64 ) as num_aliphatic_rings,
CAST(JSON_VALUE(molecular_descriptors_json, "$.NHOHCount") as INT64 ) as nhoh_count,
CAST(JSON_VALUE(molecular_descriptors_json, "$.NOCount") as INT64 ) as no_count,
CAST(JSON_VALUE(molecular_descriptors_json, "$.NumAliphaticCarbocycles") as INT64 ) as num_alpiphatic_carbocycles,
CAST(JSON_VALUE(molecular_descriptors_json, "$.NumAliphaticHeterocycles") as INT64 ) as num_aliphatic_heterocycles,
CAST(JSON_VALUE(molecular_descriptors_json, "$.NumAromaticCarbocycles") as INT64 ) as num_aromatic_carbocycles,
CAST(JSON_VALUE(molecular_descriptors_json, "$.NumAromaticHeterocycles") as INT64 ) as num_aromatic_heterocycles,
CAST(JSON_VALUE(molecular_descriptors_json, "$.NumAromaticRings") as INT64 ) as num_aromatic_rings,
CAST(JSON_VALUE(molecular_descriptors_json, "$.NumHAcceptors") as INT64 ) as num_h_acceptors,
CAST(JSON_VALUE(molecular_descriptors_json, "$.NumHDonors") as INT64 ) as num_h_donors,
CAST(JSON_VALUE(molecular_descriptors_json, "$.NumHeteroatoms") as INT64 ) as num_hetero_atoms,
CAST(JSON_VALUE(molecular_descriptors_json, "$.NumRadicalElectrons") as INT64 ) as num_radical_electrons,
CAST(JSON_VALUE(molecular_descriptors_json, "$.NumRotatableBonds") as INT64 ) as num_rotatable_bonds,
CAST(JSON_VALUE(molecular_descriptors_json, "$.NumSaturatedCarbocycles") as INT64 ) as num_saturated_carbocycles,
CAST(JSON_VALUE(molecular_descriptors_json, "$.NumSaturatedHeterocycles") as INT64 ) as num_saturated_heterocycles,
CAST(JSON_VALUE(molecular_descriptors_json, "$.NumSaturatedRings") as INT64 ) as num_saturated_rings,
CAST(JSON_VALUE(molecular_descriptors_json, "$.NumValenceElectrons") as INT64 ) as num_valence_electrons
from `exports.savi_fp_json` where fingerprint_json != "" and molecular_descriptors_json != ""
)

Query complete after 0.00s: 100%|██████████| 5/5 [00:00<00:00, 2578.89query/s]                        

25.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 2 loops each)


Substructure Search Stored Procedure explicitly queries savi_fp_clustered. Should probably be an input parameter. But for now, this call simply works

In [24]:
%%timeit -n 2 -r 1

%%bigquery df

CALL `cheminformatics_test.substructure_search`("CCCCCCCCCCCC");

Downloading: 100%|██████████| 2434/2434 [00:02<00:00, 1088.75rows/s]

15.8 s ± 0 ns per loop (mean ± std. dev. of 1 run, 2 loops each)


In [25]:
df

,smiles
0,OC1CCC=C1CCCC(CCC)O
1,C1(=CCCC1)CC3[C@H]2C[C@H](C2(C)C)C[C@@H]3O
2,O(CC=C)CCCCCCCCCCCC
3,NC1C(=CCC1)CC3[C@H]2C[C@H](C2(C)C)C[C@@H]3O
4,C[C@@H](NC(=O)C4C1C2CC5C3CC(C1C23)C45)CF
...,...
2429,COC(=O)C[S](=O)(=O)CCC(=O)OCC(CCCCCCCCCCCC)C
2430,N(CCCCCCCCCCCC(=O)O)[S](C1=CC2=C(C=C1)CC[S]2(=...
2431,N(CCCCCCCCCCCC(=O)O)[S](C1=C(C=C(C(O)=O)C=C1)C...
2432,O(C(=O)C[S](=O)(=O)C1=CC=C(C=C1)[N](=O)=O)CCC(...


In [12]:
%%bigquery df

CALL `cheminformatics_test.morgan_similarity`("C[C@@H](CC(=C)C)C(=O)OCC1[C@@H]2COC[C@H]12");

Downloading: 100%|██████████| 13/13 [00:01<00:00,  6.57rows/s]


In [13]:
df

,smiles
0,C[C@@H](CC(=C)C)C(=O)OCC1CCNC1
1,O(CC1[C@@H]2COC[C@H]12)C(C3=C(C)C=C(S3)F)=O
2,C[C@@H](CC(=C)C)C(=O)OCC1CCN(C)C1
3,O(C(=O)CC1(CC1)C2CC2)CC3[C@@H]4COC[C@H]34
4,O(C(=O)C1(CCl)CCCCC1)CC2[C@@H]3COC[C@H]23
5,O(C(=O)CN1CCCCCC1)CC2[C@@H]3COC[C@H]23
6,O(CC1[C@@H]2COC[C@H]12)C(C3=C[N](C(C)C)N=C3C)=O
7,O(C(=O)C(=O)C1CCCCC1)CC2[C@@H]3COC[C@H]23
8,CC(C)O[C@@H]1COC[C@H]1OC(C(C)CC(=C)C)=O
9,O(CC1[C@@H]2COC[C@H]12)C(CC3(CC(OC)=O)CCCC3)=O
